In [1]:
import numpy as np
from collections import Counter
import copy
import pickle

In [2]:
source_summary = []
source_tweets = []
f = open('./MainDataset/tweet_dataset/parallel_data/summaries')
for line in f:
    source_summary.append(line)

In [3]:
f.close()
f = open('./MainDataset/tweet_dataset/parallel_data/tweets')
for line in f:
    source_tweets.append(line)

Will try to train on only 10000 tweets for now


In [4]:
source_summary = source_summary[:10000]
source_tweets = source_tweets[:10000]

Exploring the Dataset

Creating the lookup tables for the source data and source tweets

In [5]:
CODES = {'<PAD>':0,'<EOS>':1,'<UNK>':2,'<GO>':3}
def createLookupTables(text):
    vocab = set(text.split())
    vocab_to_int = copy.copy(CODES)
    for v_i, v in enumerate(vocab,len(CODES)):
        vocab_to_int[v] = v_i
    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}
    return vocab_to_int, int_to_vocab

In [6]:
# A function that uses the source data vocab_to_int and source tweets vocab_to_int
# And converts them to their respective vocab ids
def text_to_ids(source_sentences,target_sentences,source_vocab_to_int,target_vocab_to_int):
    source_text_id = []
    target_text_id = []
    for i in xrange(len(source_sentences)):
        source_sentence = source_sentences[i]
        target_sentence = target_sentences[i]
        source_tokens = source_sentence.split()
        target_tokens = target_sentence.split()
        source_token_id = []
        target_token_id = []
        for index, token in enumerate(source_tokens):
            if token != "":
                source_token_id.append(source_vocab_to_int[token])
        for index, token in enumerate(target_tokens):
            if token != "":
                target_token_id.append(target_vocab_to_int[token])
        target_token_id.append(target_vocab_to_int['<EOS>'])
        source_text_id.append(source_token_id)
        target_text_id.append(target_token_id)
    return source_text_id, target_text_id

def preprocess_and_save_data(source_sentences,target_sentences,text_to_ids):
    source_text = "\n".join(source_sentences)
    target_text = "\n".join(target_sentences)
    source_text = source_text.lower()
    target_text = target_text.lower()
    source_sentences = source_text.split("\n")
    target_sentences = target_text.split("\n")
    source_vocab_to_int, source_int_to_vocab = createLookupTables(source_text)
    target_vocab_to_int, target_int_to_vocab = createLookupTables(target_text)
    source_text, target_text = text_to_ids(source_sentences,target_sentences,source_vocab_to_int,target_vocab_to_int)
    pickle.dump(((source_text, target_text),(source_vocab_to_int, target_vocab_to_int),(source_int_to_vocab, target_int_to_vocab)), open('preprocess.p', 'wb'))

    
        

In [7]:
preprocess_and_save_data(source_summary,source_tweets,text_to_ids)

Preprocessed file saved in a pickle file.

In [8]:
import pickle

def load_preprocess():
    with open('preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)

In [9]:
import numpy as np

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()

In [10]:
import tensorflow as tf

In [11]:
if not tf.test.gpu_device_name():
    print "No GPU Found! All the Best for training."
print "Tensorflow Version:",tf.__version__

No GPU Found! All the Best for training.
Tensorflow Version: 1.4.0


In [12]:
def enc_dec_model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len

In [13]:


# hyperparam_inputs function creates and returns parameters (TF placeholders) related to hyper-parameters to the model.

#     lr_rate is learning rate
#     keep_prob is the keep probability for Dropouts



def hyperparam_inputs():
    lr_rate = tf.placeholder(tf.float32, name='lr_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return lr_rate, keep_prob



In [14]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :return: Preprocessed target data
    """
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat

In [15]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(stacked_cells, 
                                       embed, 
                                       dtype=tf.float32)
    return outputs, state


In [16]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [17]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [18]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state, 
                                            cells, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state, 
                                            cells, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [19]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, enc_states = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_sentence_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output

In [20]:
display_step = 300

epochs = 13
batch_size = 128

rnn_size = 128
num_layers = 3

encoding_embedding_size = 200
decoding_embedding_size = 200

learning_rate = 0.001
keep_probability = 0.5


In [21]:
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    lr, keep_prob = hyperparam_inputs()
    
    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    # https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    # - Returns a mask tensor representing the first N positions of each cell.
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [22]:
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

In [23]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
print "Train Split Done!"
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
print "Validation Split Done!"
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    print "Session Created?!"
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        print "Epoch Train!"
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):
            print "Batch Train!"

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:
                print "Here 1"
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})
                print "Here 2"

                train_acc = get_accuracy(target_batch, batch_train_logits)
                print "Training Accuracy Found"
                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)
                print "Validation Accuracy Found"

                print "Epoch",epoch_i,"Batch",batch_i,len(source_int_text) // batch_size,"Train Accuracy",train_acc,"Validation Accuracy",valid_acc,"Loss",loss

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Train Split Done!
Validation Split Done!
Session Created?!
Epoch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch

Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Batch Train!
Epoch Train!

In [24]:
def save_params(params):
    with open('params.p', 'wb') as out_file:
        pickle.dump(params, out_file)


def load_params():
    with open('params.p', mode='rb') as in_file:
        return pickle.load(in_file)


In [25]:
save_params(save_path)

In [26]:
import tensorflow as tf
import numpy as np

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()
load_path = load_params()

In [28]:
def sentence_to_seq(sentence, vocab_to_int):
    results = []
    for word in sentence.split(" "):
        if word in vocab_to_int:
            results.append(vocab_to_int[word])
        else:
            results.append(vocab_to_int['<UNK>'])
            
    return results

translate_sentence = 'Topological quantum computing has recently proven itself to be a powerful computational model when constructing viable architectures for large scale computation. The topological model is constructed from the foundation of a error correction code, required to correct for inevitable hardware faults that will exist for a large scale quantum device. It is also a measurement based model of quantum computation, meaning that the quantum hardware is responsible only for the construction of a large, computationally universal quantum state. This quantum state is then strategically consumed, allowing for the realisation of a fully error corrected quantum algorithm. The number of physical qubits needed by the quantum hardware and the amount of time required to implement an algorithm is dictated by the manner in which this universal quantum state is consumed. In this paper we examine the problem of algorithmic optimisation in the topological lattice and introduce the required elements that will be needed when designing a classical software package to compile and implement a large scale algorithm on a topological quantum computer. '

translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [2, 40634, 27620, 575, 41510, 12533, 23956, 25370, 14007, 40566, 33150, 14043, 22116, 9588, 22541, 15869, 26710, 12452, 25401, 43955, 5313, 2, 12215, 22116, 40070, 45564, 11186, 15405, 24088, 22197, 40566, 25909, 15943, 14141, 17141, 25370, 42866, 12452, 15886, 26676, 30036, 39968, 47585, 12539, 12452, 40566, 25401, 43955, 40634, 24483, 2, 40070, 1814, 40566, 25625, 29514, 22116, 22197, 40634, 5314, 37304, 39968, 15405, 40634, 26676, 40070, 12165, 28066, 12452, 15405, 14805, 22197, 40566, 19739, 40004, 5876, 40634, 13531, 2, 40634, 41520, 40070, 15085, 35332, 24559, 11065, 12452, 15405, 32096, 22197, 40566, 16594, 25909, 1608, 40634, 20966, 2, 13647, 22197, 15556, 37162, 305, 14029, 15405, 40634, 26676, 21875, 15405, 40477, 22197, 36712, 17141, 25370, 8199, 13313, 2205, 40070, 38081, 14029, 15405, 15189, 18417, 35554, 38239, 5876, 40634, 41520, 40070, 24558, 2, 38239, 1668, 27341, 7345, 15405, 2906, 22197,